# 0. Initialisation

In [2]:
import pandas as pd
import numpy as np 
from helpers2 import S3Connection

In [3]:
s3 = S3Connection(bucket_name="clichere/diffusion")

Connection successful


# 1. Ouverture et Nettoyage des fichiers DPE

## 1.1. DPE après juillet 2021

In [3]:
path_logements_existants = "DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
DPE_logements_existants = s3.read_csv_from_s3(path_logements_existants)

In [10]:
print(DPE_logements_existants.columns)

Index(['Date_réception_DPE', 'Date_établissement_DPE', 'Modèle_DPE',
       'Date_fin_validité_DPE', 'Version_DPE', 'Méthode_application_DPE',
       'Etiquette_DPE', 'Etiquette_GES', 'Année_construction', 'Type_bâtiment',
       'Période_construction', 'Surface_habitable_logement', 'Adresse_brute',
       'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)',
       'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN',
       'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)',
       'Coordonnée_cartographique_Y_(BAN)', 'Code_postal_(brut)',
       'N°_étage_appartement', 'Nom_résidence', 'Cage_d'escalier',
       'Complément_d'adresse_logement', 'Statut_géocodage',
       'Nom__commune_(Brut)', 'N°_département_(BAN)', 'N°_région_(BAN)',
       'Complément_d'adresse_bâtiment'],
      dtype='object')


In [26]:
DPE_logements_existants_clean = DPE_logements_existants[[ 'Date_établissement_DPE',
       'Etiquette_DPE', 'Type_bâtiment', 'Année_construction',
       'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)' ,'N°_étage_appartement',
       'N°_département_(BAN)', 'N°_région_(BAN)','Code_INSEE_(BAN)', 'Version_DPE']]

In [27]:
DPE_logements_existants_clean.sample(10)

,Date_établissement_DPE,Etiquette_DPE,Type_bâtiment,Année_construction,Période_construction,Surface_habitable_logement,Adresse_(BAN),N°_étage_appartement,N°_département_(BAN),N°_région_(BAN),Code_INSEE_(BAN),Version_DPE
1697659,2021-10-11,C,appartement,1947.0,avant 1948,106.7,20 Rue Vauban 69006 Lyon,0.0,69,84.0,69386,1.0
2931255,2023-04-27,C,appartement,NaN,1948-1974,72.8,2 Avenue Paul Eluard 93000 Bobigny,0.0,93,11.0,93008,2.2
2966550,2022-03-17,E,maison,1948.0,1948-1974,152.0,8 Rue de Melbourne 16100 Cognac,0.0,16,75.0,16102,2.0
3348450,2022-09-16,D,maison,2004.0,2001-2005,96.2,51 Rue Eugène Jolly 68100 Mulhouse,0.0,68,44.0,68224,2.2
581435,2022-02-22,D,maison,1947.0,avant 1948,60.9,4 Rue Victor Hugo 59390 Lannoy,0.0,59,32.0,59332,2.1
1778155,2022-11-10,F,appartement,NaN,avant 1948,52.0,6 Impasse du Nord 78510 Triel-sur-Seine,0.0,78,11.0,78624,2.2
3567236,2021-11-30,C,maison,1900.0,avant 1948,93.0,17 Grande Rue 27700 Cuverville,NaN,27,28.0,27194,2.0
47892,2023-03-17,C,immeuble,1980.0,1978-1982,NaN,30 Avenue Mozart 44210 Pornic,NaN,44,52.0,44131,2.2
3680634,2022-06-24,C,maison,NaN,avant 1948,147.0,42 Rue du Montay 38230 Tignieu-Jameyzieu,0.0,38,84.0,38507,2.1
2355497,2023-04-14,B,appartement,NaN,1983-1988,86.0,3 Rue de l'Hortet 34500 Béziers,0.0,34,76.0,34032,2.2


In [28]:
print(DPE_logements_existants_clean.shape)

(4537525, 12)


In [29]:
doublons_DPE = DPE_logements_existants_clean.duplicated().sum()
total_obs_DPE = len(DPE_logements_existants_clean)
ratio_doublons_DPE = doublons_DPE/total_obs_DPE
print(ratio_doublons_DPE)

0.18243271386934506


Environ 18,3% des observations sont des doublons, enlevons les

In [30]:
DPE_logements_existants_clean = DPE_logements_existants_clean.drop_duplicates()

## 1.2. DPE avant juillet 2021

In [4]:
path_DPE_avant_2021 = "DPE/DPE_ADEME/dpe_avant_2021.csv"

DPE_avant_2021 = s3.read_csv_from_s3(path_DPE_avant_2021)

In [5]:
DPE_avant_2021.sample(5)

,date_etablissement_dpe,classe_consommation_energie,classe_estimation_ges,annee_construction,surface_thermique_lot,latitude,longitude,tr002_type_batiment_description,code_insee_commune_actualise,tv016_departement_code,geo_adresse
5388613,2016-08-29,N,N,1947,139.00,50.784590,1.886059,Maison Individuelle,62020,62,274 Rue du Cap Gris Nez 62850 Alembon
2804458,2016-11-28,D,E,1972,57.73,49.265018,4.019328,Logement,51454,51,22 rue Marcel Thil 51100 Reims
9486807,2018-03-12,A,B,2018,23.30,48.837797,2.314775,Logement,75117,75,Voie Cr/15 75015 Paris
6151866,2019-09-23,C,A,2008,92.00,44.249297,4.634406,Maison Individuelle,30202,30,Impasse des Hauts de Saint-Joseph 30130 Pont-S...
6503372,2017-06-14,C,D,2006,59.20,48.176999,-1.661033,Bâtiment collectif à usage principal d'habitation,35024,35,1c Rue de Barberino Di Mugello 35830 Betton


In [8]:
DPE_avant_2021['Etiquette_DPE'] = DPE_avant_2021[['classe_consommation_energie', 'classe_estimation_ges']].max(axis=1)

In [19]:

DPE_avant_2021 = DPE_avant_2021.rename(columns={'date_etablissement_dpe': 'Date_établissement_DPE', 
'annee_construction' : 'Année_construction', 'surface_thermique_lot' : 'Surface_habitable_logement',
'tr002_type_batiment_description' : 'Type_bâtiment', 'code_insee_commune_actualise' : 'Code_INSEE_(BAN)',
'geo_adresse' : 'Adresse_(BAN)', 'tv016_departement_code' : 'N°_département_(BAN)'})

In [20]:
DPE_avant_2021.sample(10)

,Date_établissement_DPE,classe_consommation_energie,classe_estimation_ges,Année_construction,Surface_habitable_logement,latitude,longitude,Type_bâtiment,Code_INSEE_(BAN),N°_département_(BAN),Adresse_(BAN),Etiquette_DPE
10442863,2019-11-19,D,E,1975,19.46,50.373368,3.088175,appartement,59178,59,Rue Leon Escoffier 59500 Douai,E
2477405,2014-09-24,N,N,1977,25.00,48.785032,2.316280,appartement,92014,92,32 Boulevard du Maréchal Joffre 92340 Bourg-la...,N
2589811,2016-09-10,E,B,1940,93.00,44.753978,5.368515,maison,26113,26,10bis Rue Anglaise 26150 Die,E
801518,2014-04-23,E,C,2014,94.00,NaN,NaN,maison,45043,45,NaN,E
7367776,2020-08-11,N,N,1947,71.00,46.913724,-2.045300,maison,85018,85,17 Rue des sables 85230 Beauvoir-sur-Mer,N
5580816,2016-08-02,G,G,1974,51.00,48.939589,2.019103,maison,78123,78,383 Rue Pasteur 78955 Carrières-sous-Poissy,G
7463346,2018-01-31,D,B,1989,39.15,43.170292,6.530032,appartement,83036,83,Rue de Verdun 83240 Cavalaire-sur-Mer,D
10237777,2019-12-04,G,G,1948,12.27,48.832442,2.303361,appartement,75115,75,67 Rue Brancion 75015 Paris,G
1980268,2014-06-20,D,F,1960,147.00,43.324842,6.656132,appartement,83115,83,9 Avenue Alphonse Daudet 83120 Sainte-Maxime,F
9448873,2019-04-03,A,B,2017,114.00,43.603105,1.251494,maison,31291,31,Route de Pibrac 31490 Léguevin,B


In [21]:
print(DPE_avant_2021["Type_bâtiment"].value_counts(dropna=False))

Type_bâtiment
appartement    5290076
maison         4568519
immeuble        870355
Name: count, dtype: int64


In [18]:
DPE_avant_2021['Type_bâtiment'] = DPE_avant_2021['Type_bâtiment'].replace({
    'Logement': 'appartement',
    'Maison Individuelle': 'maison',
    "Bâtiment collectif à usage principal d'habitation" : "immeuble"
})


In [31]:
print(DPE_logements_existants_clean.columns)
print(DPE_avant_2021.columns)

Index(['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment',
       'Année_construction', 'Période_construction',
       'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_étage_appartement',
       'N°_département_(BAN)', 'N°_région_(BAN)', 'Code_INSEE_(BAN)',
       'Version_DPE'],
      dtype='object')
Index(['Date_établissement_DPE', 'classe_consommation_energie',
       'classe_estimation_ges', 'Année_construction',
       'Surface_habitable_logement', 'latitude', 'longitude', 'Type_bâtiment',
       'Code_INSEE_(BAN)', 'N°_département_(BAN)', 'Adresse_(BAN)',
       'Etiquette_DPE'],
      dtype='object')


In [128]:
outpath1 = 'DPE/DPE_après_2021.parquet'

s3.from_pandas_to_parquet_store_in_s3(DPE_logements, outpath1)

Fichier Parquet écrit avec succès dans DPE/DPE_après_2021.parquet


# 2. Ouverture et nettoyage des fichiers valeurs foncières

## 2.1. Téléchargement et fusion des fichiers csv

In [4]:
path_vf_2024 = "valeursfoncieres/vf_2024.csv"
vf_2024 = s3.read_csv_from_s3(path_vf_2024)

path_vf_2023 = "valeursfoncieres/vf_2023.csv"
vf_2023 = s3.read_csv_from_s3(path_vf_2023)

path_vf_2022 = "valeursfoncieres/vf_2022.csv"
vf_2022 = s3.read_csv_from_s3(path_vf_2022)

path_vf_2021 = "valeursfoncieres/vf_2021.csv"
vf_2021 = s3.read_csv_from_s3(path_vf_2021)

path_vf_2020 = "valeursfoncieres/vf_2020.csv"
vf_2020 = s3.read_csv_from_s3(path_vf_2020)

path_vf_2019 = "valeursfoncieres/vf_2019.csv"
vf_2019 = s3.read_csv_from_s3(path_vf_2019)

In [6]:
print(vf_2019["valeur_fonciere"].describe())
print(vf_2020["valeur_fonciere"].describe())
print(vf_2021["valeur_fonciere"].describe())
print(vf_2022["valeur_fonciere"].describe())
print(vf_2023["valeur_fonciere"].describe())
print(vf_2024["valeur_fonciere"].describe())
print(vf_2019.columns)

count    1.945453e+06
mean     1.006466e+06
std      8.090271e+06
min      1.000000e-02
25%      6.450000e+04
50%      1.550000e+05
75%      2.821490e+05
max      8.450000e+08
Name: valeur_fonciere, dtype: float64
count    3.471436e+06
mean     8.727544e+05
std      7.780260e+06
min      1.000000e-02
25%      6.655000e+04
50%      1.550000e+05
75%      2.800000e+05
max      4.900000e+08
Name: valeur_fonciere, dtype: float64
count    4.629636e+06
mean     2.006912e+06
std      2.036543e+07
min      1.000000e-01
25%      7.250000e+04
50%      1.660000e+05
75%      3.060000e+05
max      1.415000e+10
Name: valeur_fonciere, dtype: float64
count    4.635517e+06
mean     2.801775e+06
std      2.677626e+07
min      1.000000e-02
25%      7.600000e+04
50%      1.750000e+05
75%      3.200000e+05
max      1.003401e+09
Name: valeur_fonciere, dtype: float64
count    3.765454e+06
mean     6.174104e+05
std      5.111104e+06
min      1.000000e-01
25%      6.156900e+04
50%      1.600000e+05
75%      3.0

In [5]:
vf_new = pd.concat([vf_2024, vf_2023, vf_2022, vf_2021, vf_2020, vf_2019], ignore_index=True)


In [8]:
vf_new.sample(20)
print(vf_new.shape)
print(vf_new.columns)
print(vf_new["date_mutation"].value_counts)
doublons_vf_2024 = vf_new.duplicated()
print(f"nombre de doublons {doublons_vf_2024.sum()}")

(20195667, 40)
Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')
<bound method IndexOpsMixin.value_counts of 0           2024-01-02
1           2024-01-03
2           2024-01

In [26]:
print(vf_new["lot1_surface_carrez"].value_counts(dropna=False))
print(vf_new["lot2_surface_carrez"].value_counts(dropna=False))
print(vf_new["surface_reelle_bati"].value_counts(dropna=False))

lot1_surface_carrez
NaN       18442001
12.50         6410
12.00         3048
15.00         2285
10.00         1799
            ...   
390.13           1
278.46           1
1.17             1
169.99           1
203.22           1
Name: count, Length: 28348, dtype: int64
lot2_surface_carrez
NaN       19651067
65.00          445
60.00          436
70.00          417
68.00          403
            ...   
348.42           1
318.51           1
7.48             1
178.45           1
185.47           1
Name: count, Length: 17798, dtype: int64
surface_reelle_bati
NaN         13019863
80.0          141525
60.0          134169
70.0          126882
90.0          124531
              ...   
39667.0            1
78354.0            1
27057.0            1
107476.0           1
15318.0            1
Name: count, Length: 9346, dtype: int64


In [10]:
vf_new_clean = vf_new[['date_mutation', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'code_postal', 'code_commune',
       'nom_commune', 'type_local', 'code_departement',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'surface_terrain', 'longitude', 'latitude']]

In [11]:
vf_new_clean['Adresse'] = vf_new_clean['adresse_numero'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf_new_clean['adresse_nom_voie'].fillna('').astype(str) + " " + \
                vf_new_clean['code_postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf_new_clean['nom_commune'].fillna('').astype(str)

vf_new_clean['Adresse'] = vf_new_clean['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

/tmp/ipykernel_10291/1243246230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vf_new_clean['Adresse'] = vf_new_clean['adresse_numero'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
/tmp/ipykernel_10291/1243246230.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vf_new_clean['Adresse'] = vf_new_clean['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides


In [8]:
vf_new_clean.sample(10)

,date_mutation,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,code_postal,code_commune,nom_commune,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,Adresse
18029789,2020-03-24,Vente,242000.0,1.0,NaN,RUE GAMBETTA,93330.0,93050,Neuilly-sur-Marne,Appartement,77.0,4.0,NaN,2.532434,48.856705,1 RUE GAMBETTA 93330 Neuilly-sur-Marne
16226940,2020-12-21,Vente,40125.0,NaN,NaN,LE BICETRE,42660.0,42172,Planfoy,NaN,NaN,NaN,160.0,4.439415,45.380026,LE BICETRE 42660 Planfoy
5079738,2023-03-20,Vente,284100.0,1.0,NaN,RUE HEBERT,92140.0,92023,Clamart,Appartement,40.0,2.0,NaN,2.272463,48.813371,1 RUE HEBERT 92140 Clamart
8467828,2022-09-28,Vente,95000.0,5.0,NaN,RUE BARANDE,66000.0,66136,Perpignan,Maison,59.0,4.0,77.0,2.890894,42.710178,5 RUE BARANDE 66000 Perpignan
11734755,2021-03-20,Vente,592000.0,NaN,NaN,LA LANDE DES PLANCHES,35300.0,35115,Fougères,NaN,NaN,NaN,1760.0,-1.212693,48.333207,LA LANDE DES PLANCHES 35300 Fougères
5150835,2023-12-29,Vente,195150.0,74.0,NaN,BD JEAN JAURES,93400.0,93070,Saint-Ouen-sur-Seine,Appartement,24.0,1.0,NaN,2.334111,48.912604,74 BD JEAN JAURES 93400 Saint-Ouen-sur-Seine
19028370,2019-07-31,Vente,463000.0,37.0,NaN,AV DES CANDELONS,40510.0,40296,Seignosse,Maison,57.0,3.0,668.0,-1.426590,43.705827,37 AV DES CANDELONS 40510 Seignosse
17550821,2020-09-23,Vente,95000.0,81.0,NaN,RUE LEDAY,80100.0,80001,Abbeville,Maison,76.0,3.0,82.0,1.847235,50.111517,81 RUE LEDAY 80100 Abbeville
5473854,2022-08-19,Vente,1409.0,NaN,NaN,LA GARDE,3450.0,03311,Vicq,NaN,NaN,NaN,1629.0,3.080249,46.160290,LA GARDE 3450 Vicq
11445464,2021-06-25,Adjudication,49000.0,NaN,NaN,LA BARTHE,31230.0,31494,Saint-Laurent,NaN,NaN,NaN,2490.0,0.796677,43.334259,LA BARTHE 31230 Saint-Laurent


In [12]:
doublons_vf_new = vf_new_clean.duplicated().sum()
nombre_obs_vf_new = len(vf_new_clean)
ratio_vf_new = doublons_vf_new/nombre_obs_vf_new

print(f"Part de doublons dans le dataframe : {ratio_vf_new}")

Part de doublons dans le dataframe : 0.12893948984205375


Environ 13% des observations sont en doublon

In [13]:
vf_final = vf_new_clean.drop_duplicates()

In [14]:
print(vf_final.shape)

(17591648, 17)


In [16]:
outpath_vf_new = 'valeursfoncieres/vf_new.parquet'

s3.from_pandas_to_parquet_store_in_s3(vf_final, outpath_vf_new)

Fichier Parquet écrit avec succès dans valeursfoncieres/vf_new.parquet


## 2.2. Téléchargement et fusion des fichiers txt

In [3]:
dtype_spec = {"Code commune": str}

path_valeursfoncieres_2022 = "Valeursfoncières/valeursfoncieres-2022.txt"
vf2022 = s3.read_txt_from_s3(path_valeursfoncieres_2022, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2021 = "Valeursfoncières/valeursfoncieres-2021.txt"
vf2021 = s3.read_txt_from_s3(path_valeursfoncieres_2021, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2020 = "Valeursfoncières/valeursfoncieres-2020.txt"
vf2020 = s3.read_txt_from_s3(path_valeursfoncieres_2020, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2019 = "Valeursfoncières/valeursfoncieres-2019.txt"
vf2019 = s3.read_txt_from_s3(path_valeursfoncieres_2019, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2018 = "Valeursfoncières/valeursfoncieres-2018.txt"
vf2018 = s3.read_txt_from_s3(path_valeursfoncieres_2018, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2017 = "Valeursfoncières/valeursfoncieres-2017.txt"
vf2017 = s3.read_txt_from_s3(path_valeursfoncieres_2017, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2016 = "Valeursfoncières/valeursfoncieres-2016.txt"
vf2016 = s3.read_txt_from_s3(path_valeursfoncieres_2016, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2015 = "Valeursfoncières/valeursfoncieres-2015.txt"
vf2015 = s3.read_txt_from_s3(path_valeursfoncieres_2015, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2014 = "Valeursfoncières/valeursfoncieres-2014.txt"
vf2014 = s3.read_txt_from_s3(path_valeursfoncieres_2014, sep="|", dtype_spec=dtype_spec)


In [4]:
print(vf2014.shape)
print(vf2015.shape)
print(vf2016.shape)
print(vf2017.shape)
print(vf2018.shape)
print(vf2019.shape)
print(vf2020.shape)
print(vf2021.shape)
print(vf2022.shape)

(2516688, 43)
(2749830, 43)
(2936524, 43)
(3383622, 43)
(3335292, 43)
(3625903, 43)
(3514698, 43)
(4649209, 43)
(3803885, 43)


In [5]:
vf = pd.concat([vf2022, vf2021, vf2020, vf2019, vf2018, vf2017, vf2016, vf2015, vf2014], ignore_index=True)


In [6]:
print(vf.columns)

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Code service CH'],
      dtype='object')


In [8]:
print(vf["Nature mutation"].value_counts(dropna=False))
print(vf["1er lot"].value_counts(dropna=False))

Nature mutation
Vente                                 27775517
Vente en l'état futur d'achèvement     2085142
Echange                                 405566
Adjudication                            111197
Vente terrain à bâtir                    96519
Expropriation                            41710
Name: count, dtype: int64
1er lot
NaN      21035867
1          326170
2          268312
3          210252
10         179842
           ...   
33K             1
20805           1
7372            1
7644            1
7700            1
Name: count, Length: 22126, dtype: int64


In [9]:
vf_clean = vf[['Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'Type de voie', 'Voie', 'Code postal',
       'Commune', 'Code commune', 
       'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot',
       'Surface Carrez du 3eme lot',
       'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot',
       'Type local','Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain']]

In [12]:
vf_clean[vf_clean["Type local"] == "Appartement"].sample(10)

,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Voie,Code postal,Commune,Code commune,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
22385966,29/02/2016,Vente,"145000,00",5250.0,NaN,LE VILLAGE,4150.0,BANON,18,NaN,NaN,NaN,NaN,NaN,Appartement,32.0,2.0,S,NaN,130.0
22258238,06/04/2017,Vente,"170000,00",77.0,RUE,TAITBOUT,75009.0,PARIS 09,109,"19,52",NaN,NaN,NaN,NaN,Appartement,25.0,1.0,NaN,NaN,NaN
5342718,28/10/2021,Vente,"75000,00",25.0,RUE,JEAN JAURES,33230.0,COUTRAS,138,"39,30",NaN,NaN,NaN,NaN,Appartement,40.0,2.0,NaN,NaN,NaN
18650803,26/04/2018,Vente,"684000,00",124.0,BD,DE LA REPUBLIQUE,92210.0,SAINT-CLOUD,64,"132,53",NaN,NaN,NaN,NaN,Appartement,130.0,6.0,NaN,NaN,NaN
14727776,01/10/2019,Vente,"117000,00",1.0,BD,ETIENNE HARDY,77610.0,FONTENAY-TRESIGNY,192,"34,79",NaN,NaN,NaN,NaN,Appartement,51.0,2.0,NaN,NaN,NaN
184050,24/05/2022,Vente,"212000,00",102.0,AV,DE GRASSE,6800.0,CAGNES SUR MER,27,NaN,"65,04",NaN,NaN,NaN,Appartement,65.0,3.0,NaN,NaN,NaN
11401105,20/11/2020,Vente,"124000,00",61.0,AV,DE CLARET,83000.0,TOULON,137,"42,07",NaN,NaN,NaN,NaN,Appartement,42.0,2.0,NaN,NaN,NaN
1680133,20/06/2022,Vente,"200020,00",12.0,RUE,MARIE ANNE DU BOCCAGE,44000.0,NANTES,109,"32,35",NaN,NaN,NaN,NaN,Appartement,32.0,2.0,NaN,NaN,NaN
24719641,01/04/2016,Adjudication,"76000,00",5.0,RUE,HENRI FRENAY,83120.0,SAINTE MAXIME,115,NaN,NaN,NaN,NaN,NaN,Appartement,20.0,1.0,NaN,NaN,NaN
27788377,21/09/2015,Adjudication,"290000,00",35.0,RUE,FERNAND PELLOUTIER,92100.0,BOULOGNE-BILLANCOURT,12,NaN,NaN,NaN,NaN,NaN,Appartement,60.0,3.0,NaN,NaN,NaN


In [ ]:
doublons4 = vf_clean.duplicated().sum()
total_obs4 = len(vf_clean)
ratio_doublons4 = doublons4/total_obs4
print(ratio_doublons4)

In [19]:
outpath2 = 'Valeursfoncières/vf.parquet'

s3.from_pandas_to_parquet_store_in_s3(vf_clean, outpath2)

Fichier Parquet écrit avec succès dans Valeursfoncières/vf.parquet


### Normalisation des adresses

In [ ]:
# Convertir explicitement toutes les colonnes en chaînes
#vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
 #               vf['Type de voie'].fillna('').astype(str) + " " + \
 #               vf['Voie'].fillna('').astype(str) + ", " + \
 #               vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
 #               vf['Commune'].fillna('').astype(str)
#print(vf.describe)

In [ ]:
#vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides
#print(vf2022[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune', 'Adresse']].head())

In [ ]:
#vf2022['Adresse_Normalisee'] = vf2022['Adresse'].apply(s3.normalize_address)

#print(vf2022[['Adresse', 'Adresse_Normalisee']].head())

TypeError: s3_connection.normalize_address() takes 1 positional argument but 2 were given